# Modelos Lineales Regularizados

En muchas técnicas de aprendizaje automático, el aprendizaje consiste en encontrar los coeficientes que minimizan una función de coste. La regularización consiste en añadir una penalización a la función de coste. Esta penalización produce modelos más simples que generalizan mejor.

Cuando usamos regularización, añadimos un término que penaliza la complejidad del modelo. En el caso del MSE, tenemos:

$$
J = MSE + \lambda \times C
$$

- C es la medida de complejidad del modelo. Dependiendo de cómo midamos la complejidad, tendremos distintos tipos de regularización. 
- λ indica cómo de importante es para nosotros que el modelo sea simple en relación a cómo de importante es su rendimiento.

Cuando usamos regularización minimizamos la complejidad del modelo a la vez que minimizamos la función de coste. Esto resulta en modelos más simples que tienden a generalizar mejor.

---
- Cuando utilices los regulizadores, debes asegurarte de que todas las características tengan una escala similar (por ejemplo, usando la clase `StandardScaler` de **Scikit-Learn**); de lo contrario, el algoritmo tardará mucho más tiempo en converger.

    ```python
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    ```
---

## Regularizacion Lasso (L1)

La Regresión Lasso es una versión regularizada de la Regresión Lineal: se añade un término de regularización igual a $\alpha \sum_{i=1}^{n} |\theta_i|$ a la función de costo.

La funcion de costo de la regresion ridge es:

$$
J(\theta) = MSE(\theta) + \alpha \sum_{i=1}^{n} |\theta_i|
$$

El hiperparámetro $\alpha$ controla cuánto deseas regularizar el modelo. Si $\alpha = 0$, entonces la Regresión Ridge es simplemente una Regresión Lineal. Si $\alpha$ es muy grande, entonces todos los pesos terminan muy cerca de cero y el resultado es una línea plana que pasa por la media de los datos.

La función de costo de Lasso no es diferenciable en $\theta_i = 0$ (para $i = 1, 2, \dots, n$), pero el Descenso de Gradiente sigue funcionando bien si utilizas en su lugar un vector de subgradiente $g$ cuando cualquier $\theta_i = 0$

$$
g(\theta, \lambda) = \nabla_\theta \text{MSE}(\theta) + \alpha 
\begin{pmatrix}
\text{sign}(\theta_1) \\
\text{sign}(\theta_2) \\
\vdots \\
\text{sign}(\theta_n)
\end{pmatrix}
$$

donde

$$
\text{sign}(\theta_i) =
\begin{cases}
-1 & \text{si } \theta_i < 0 \\
0 & \text{si } \theta_i = 0 \\
+1 & \text{si } \theta_i > 0
\end{cases}
$$

#### Cuándo usar Ridge
- Lasso es muy útil cuando sospechamos que varios de los atributos son irrelevantes.
- Algunos de los coeficientes acabarán valiendo 0, filtrando así atributos irrelevantes.


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X  + np.random.rand(100, 1)

lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)

print("Coeficientes:", lasso_reg.coef_)
print("Intercepto:", lasso_reg.intercept_)
print(f"La prediccion es:{lasso_reg.predict([[1.5]])}")

Coeficientes: [2.5159679]
Intercepto: [5.04734379]
La prediccion es:[8.82129564]


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X  + np.random.rand(100, 1)

sgd_reg = SGDRegressor(penalty="l1")
sgd_reg.fit(X, y)

print("Coeficientes:", sgd_reg.coef_)
print("Intercepto:", sgd_reg.intercept_)
print(f"La prediccion es:{sgd_reg.predict([[1.5]])}")

Coeficientes: [3.33167404]
Intercepto: [4.09808535]
La prediccion es:[9.09559641]


c:\Users\Jharv\OneDrive\Documentos\Cursos\learning-notes\env\Lib\site-packages\sklearn\utils\validation.py:1352: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Regularizacion Ridge (L2)

La Regresión Ridge (también llamada regularización de Tikhonov) es una versión regularizada de la Regresión Lineal: se añade un término de regularización igual a $\alpha \sum_{i=1}^{n} \theta_i^2$ a la función de costo.

La funcion de costo de la regresion ridge es:

$$
J(\theta) = MSE(\theta) + \alpha \frac{1}{2} \sum_{i=1}^{n} \theta_i^2
$$

El hiperparámetro $\alpha$ controla cuánto deseas regularizar el modelo. Si $\alpha = 0$, entonces la Regresión Ridge es simplemente una Regresión Lineal. Si $\alpha$ es muy grande, entonces todos los pesos terminan muy cerca de cero y el resultado es una línea plana que pasa por la media de los datos.

Para la actualizacion de la gradienta, seria sencillo ya que solamente se aumentaria un termino ya que la derivada de $\frac{1}{2} \alpha \mathbf{w}^2$ con respecto a $\mathbf{w}$ es simplemente $\alpha \mathbf{w}$, donde $\mathbf{w}$ es el vector que contiene todos los parámetros excepto el primero: $\mathbf{w} = [\theta_1, \theta_2, \dots, \theta_n]^T$.

$$
\nabla_{\theta} J(\theta) = \text{Gradiente del MSE} + \alpha \mathbf{w}
$$

Al igual que con la Regresión Lineal, podemos realizar la Regresión Ridge ya sea calculando una ecuación de forma cerrada o mediante el Descenso de Gradiente. Los pros y los contras son los mismos. La siguiente ecuacion muestra la solución de forma cerrada (donde $A$ es la matriz identidad de $(n + 1) \times (n + 1)$, excepto con un 0 en la fila superior y columna izquierda, correspondiente al término de sesgo).

$$
\hat{\theta} = (X^T X + \alpha A)^{-1} X^T y
$$

#### Cuándo usar Ridge
- Cuando dos o más variables explican casi lo mismo (alta colinealidad), el modelo lineal normal tiende a tener coeficientes muy grandes e inestables ($x_2 = 2x_1$).
- Cuando el número de características es grande o comparable al número de muestras ($n = m$), el modelo puede sobreajustar fácilmente.

### Implementacion con sklearn

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X  + np.random.rand(100, 1)

ridge_reg = Ridge(alpha=1, solver="cholesky")
ridge_reg.fit(X, y)

print("Coeficientes:", ridge_reg.coef_)
print("Intercepto:", ridge_reg.intercept_)
print(f"La prediccion es:{ridge_reg.predict([[1.5]])}")


Coeficientes: [2.9027427]
Intercepto: [4.56236206]
La prediccion es:[8.91647611]


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X  + np.random.rand(100, 1)

sgd_reg = SGDRegressor(penalty="l2")
sgd_reg.fit(X, y)

print("Coeficientes:", sgd_reg.coef_)
print("Intercepto:", sgd_reg.intercept_)
print(f"La prediccion es:{sgd_reg.predict([[1.5]])}")


Coeficientes: [3.31672044]
Intercepto: [4.00375776]
La prediccion es:[8.97883842]


c:\Users\Jharv\OneDrive\Documentos\Cursos\learning-notes\env\Lib\site-packages\sklearn\utils\validation.py:1352: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Elastic Net

Elastic Net es un punto medio entre la Regresión Ridge y la Regresión Lasso. El término de regularización es una mezcla simple de los términos de regularización de Ridge y Lasso, y puedes controlar la relación de mezcla $r$. Cuando $r = 0$, Elastic Net es equivalente a la Regresión Ridge, y cuando $r = 1$, es equivalente a la Regresión Lasso

$$
J(\theta) = \text{MSE}(\theta) + r\alpha \sum_{i=1}^{n} |\theta_i| + \frac{1 - r}{2} \alpha \sum_{i=1}^{n} \theta_i^2
$$

Para la actualizacion de la gradiente, seria sumar los terminos de la gradiente de lasso y ridge, como la siguiente formmula:

$$
g = \nabla_{\theta} \text{MSE}(\theta) + r\alpha \cdot \text{sign}(\mathbf{w}) + (1 - r)\alpha \mathbf{w}
$$

Entonces, ¿cuándo deberías usar la Regresión Lineal simple (es decir, sin ninguna regularización), Ridge, Lasso o Elastic Net? Casi siempre es preferible tener al menos un poco de regularización, por lo que, en general, deberías evitar la Regresión Lineal simple.

Ridge es un buen valor predeterminado, pero si sospechas que solo unas pocas características son realmente útiles, deberías preferir Lasso o Elastic Net, ya que tienden a reducir los pesos de las características inútiles a cero. En general, Elastic Net se prefiere sobre Lasso, ya que Lasso puede comportarse de forma errática cuando el número de características es mayor que el número de instancias de entrenamiento o cuando varias características están fuertemente correlacionadas.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X  + np.random.rand(100, 1)

elastic_reg = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_reg.fit(X, y)

print("Coeficientes:", elastic_reg.coef_)
print("Intercepto:", elastic_reg.intercept_)
print(f"La prediccion es:{elastic_reg.predict([[1.5]])}")

Coeficientes: [2.53396632]
Intercepto: [4.9576093]
La prediccion es:[8.75855878]
